In [1]:
from pdf2image import convert_from_path
import cloudinary
import cloudinary.uploader
import os

def pdf_to_images_and_upload(pdf_path):
    # Cấu hình Cloudinary bằng các biến môi trường
    cloudinary.config(
        cloud_name="dfatgrcng",
        api_key="835866625462259",
        api_secret="qMX2WoR9nn7OWmGVvbK5aYRwP8o"
    )

    # Chuyển đổi pdf thành hình ảnh
    images = convert_from_path(pdf_path)
    image_links = []
    for page_num, image in enumerate(images):
        image_filename = f"page_{page_num + 1}.png"
        image.save(image_filename, 'JPEG')
        # Tải hình ảnh lên Cloudinary
        result = cloudinary.uploader.upload(image_filename)
        image_links.append(result['secure_url'])
        # Xóa hình ảnh sau khi tải lên
        os.remove(image_filename)
    return image_links

In [2]:
import os
from dotenv import load_dotenv
from together import Together


def extract_text_with_prompt_from_image(image_url,prompt):
    # Lấy API key từ biến môi trường và cấu hình Google Gemini API
    api_key = "28fe2cf1289992f74fff42816b194e166513bf2d8dce2c0228dce23527d0df65"
    client = Together(api_key=api_key)
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
        messages=[
            {
                    "role": "user",
                    "content": [
                            {
                                    "type": "text",
                                    "text": prompt if prompt else "Extract the text from the image"
                            },
                            {
                                    "type": "image_url",
                                    "image_url": {
                                            "url": image_url
                                    }
                            }
                    ]
            }
    ],
        max_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>","<|eom_id|>"],
    )
    # print('response',response)
    return response.choices[0].message.content if response.choices else "Unable to process the image with the provided prompt."




In [3]:
from PyPDF2 import PdfReader

def get_pdf_text(pdf_docs):

    text = ""

    # Check if pdf_docs is a list or a string
    if not isinstance(pdf_docs, list):
        pdf_docs = [pdf_docs] # Convert to a list if it's a string

    # Thử đọc từng file PDF trong danh sách
    for pdf in pdf_docs:
        try:
            pdf_reader = PdfReader(pdf)  # pdf là đường dẫn của từng file PDF
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
        except IsADirectoryError: # Handle the case where the input is a directory
            print(f"Error: {pdf} is a directory, not a PDF file.")
        except FileNotFoundError: # Handle the case where the file doesn't exist
            print(f"Error: {pdf} not found.")
        except PyPDF2.errors.PdfReadError: # Handle the case where the file is not a valid PDF
            print(f"Error: {pdf} is not a valid PDF file.")

        # Nếu văn bản trả về rỗng, chuyển file PDF thành ảnh và trích xuất văn bản từ ảnh
        if not text.strip():
            image_urls = pdf_to_images_and_upload(pdf)  # Chuyển đổi từng file PDF thành ảnh
            prompt = """Trích xuất văn bản từ hình ảnh. Chỉ trả về văn bản, không cần giải thích."""
            for image_url in image_urls:
                response = extract_text_with_prompt_from_image(image_url, prompt)
                text += response.strip() + "\n\n"
    
    return text

In [4]:
get_pdf_text(["C:/Users/PC/Desktop/Slide/data/data/PL1.VPSAS.01.pdf"])

'Phụ lục số 01 \n \nCHUẨN MỰC KẾ TOÁN CÔNG VIỆT NAM SỐ 01 \nTRÌNH BÀY BÁO CÁO TÀI CHÍNH \n(Kèm theo Quyết định số 1676/QĐ-BTC ngày 01/09/2021 của Bộ Tài chính)   \n \nGIỚI THIỆU     \nHệ thống chuẩn mực kế toán công Việt Nam được Ban soạn thảo chuẩn mực kế toán \ncông thuộc Bộ Tài chính nghiên cứu, xây dựng đảm bảo đáp ứng yêu cầu tuân thủ theo \ncác thông lệ quốc tế về kế toán và phù hợp với điều kiện thực tế của Việt Nam. Các \nchuẩn mực kế toán công Việt Nam có cùng ký hiệu chuẩn mực với chuẩn mực kế toán \ncông quốc tế tương ứng.  \n \nChuẩn mực kế toán công Việt Nam (VPSAS) số 01  “Trình bày báo cáo tài chính” \nđược soạn thảo dựa trên Chuẩn mực kế toán công quốc tế (IPSAS) số 1 “Trình bày \nBáo cáo tài chính” và các quy định hiện hành về cơ chế tài chính, ngân sách của Việt \nNam. Chuẩn mực kế toán công Việt Nam số 01  quy định những nội dung phù hợp với \ncác quy định pháp lý của Việt Nam hiện hành và các quy định dự kiến có thể được sửa \nđổi, bổ sung trong thời gian tới. Chuẩn